In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
#Get all of the links from this webpage and save them in a list
url="http://web.mta.info/developers/turnstile.html"
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))


In [3]:
#Filter the list to only show links pointing to turnstile data for years 2019-2022

start19='data/nyct/turnstile/turnstile_19'
start20='data/nyct/turnstile/turnstile_20'
start21='data/nyct/turnstile/turnstile_21'
start22='data/nyct/turnstile/turnstile_22'
data_links=[]
for link in links:
    if link:
        if link.startswith(start19) or link.startswith(start20) or link.startswith(start21) or link.startswith(start22):
            data_links.append('http://web.mta.info/developers/'+link)
            
data_links.append('http://web.mta.info/developers/data/nyct/turnstile/turnstile_181229.txt')
data_links.reverse()



In [53]:
#Download the links to a temporary pandas dataframe and save these as csv files
for link in data_links:
    print(counter)
    data=pd.read_csv(link)
    data.to_csv("raw_csv_mta/"+str(counter)+".csv", index=False)

#File to lookup complex_number for each station    
rcl=pd.read_csv("https://raw.githubusercontent.com/qri-io/data-stories-scripts/master/nyc-turnstile-counts/lookup/remote_complex_lookup.csv", index_col=False)
rcl.columns = [column.strip() for column in rcl.columns]
rcl.to_csv("rcl.csv", index=False)

##This downloads all the chosen datafiles, which we then put to HDFS by running the put command in dataproc terminal


In [1]:
### Pandas only cleaning, this is not being used anymore
"""cleaned_data=pd.DataFrame()
counter = 1
for link in data_links:
    print("Processing Data from Link Number:",counter)
    data=pd.read_csv(link)
    data.columns = [column.strip() for column in data.columns]
    data["DATE_TIME"] = pd.to_datetime(data.DATE + " " + data.TIME, format="%m/%d/%Y %H:%M:%S")
    data.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)
    data = data.drop(["DESC","LINENAME","DIVISION"], axis=1, errors="ignore")
    data = data.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).last().reset_index()
    data=data.merge(rcl[['complex_id', 'remote']], how = 'left', left_on = 'UNIT', right_on = 'remote').drop_duplicates().reset_index(drop=True)
    data['DATE'] = pd.to_datetime(data['DATE_TIME']).dt.date
    data = data.drop(["remote", "TIME", "DATE_TIME"], axis=1, errors="ignore")
    cleaned_data= cleaned_data.append(data)
    counter+=1

cleaned_data[["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"]] = (cleaned_data.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES", "EXITS"].transform(lambda grp: grp.shift(1)))
cleaned_data.to_csv("intermediate_cleaned_mta.csv",index=False)

"""

'cleaned_data=pd.DataFrame()\ncounter = 1\nfor link in data_links:\n    print("Processing Data from Link Number:",counter)\n    data=pd.read_csv(link)\n    data.columns = [column.strip() for column in data.columns]\n    data["DATE_TIME"] = pd.to_datetime(data.DATE + " " + data.TIME, format="%m/%d/%Y %H:%M:%S")\n    data.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)\n    data = data.drop(["DESC","LINENAME","DIVISION"], axis=1, errors="ignore")\n    data = data.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).last().reset_index()\n    data=data.merge(rcl[[\'complex_id\', \'remote\']], how = \'left\', left_on = \'UNIT\', right_on = \'remote\').drop_duplicates().reset_index(drop=True)\n    data[\'DATE\'] = pd.to_datetime(data[\'DATE_TIME\']).dt.date\n    data = data.drop(["remote", "TIME", "DATE_TIME"], axis=1, errors="ignore")\n    cleaned_data= cleaned_data.append(data)\n    counter+=1\n'